## Setup

In [1]:
import pandas as pd
import datetime as dt
from msci.bdt import create_unique_export_set_name
from msci.bdt.context.ServiceClient import ServiceClient

url = 'https://www.barraone.com'
user_id = 'ykwon'
password = 'passive23456'
client_id = 'wu8cegzksh'

## Report setting

In [2]:
model = 'KRE3L'
port_name= '10003'
port_owner = 'ykwon'
date1 = '2024-07-16'

## Prepare the database for storing exposures

In [3]:
import sqlite3

conn = sqlite3.connect("exposures.db")
cur = conn.cursor()

In [ ]:
cur.execute("""CREATE TABLE IF NOT EXISTS exposures (
  Date Date,
  AssetID text,
  AssetIDType text,
  Factor text,
  Exposure real,
  PRIMARY KEY (Date, AssetID, Factor)
  ) """)
conn.commit()

#cur.execute('DROP TABLE exposures')
#conn.commit()

## Retrieve reports

In [4]:
with ServiceClient(url, user_id, password, client_id) as bdt_service_client:
    name = create_unique_export_set_name(portfolio=port_name, model=model)
    portfolio_list = [(port_name, port_owner)]  # should be a list of (portfolio, owner) tuples
    report_list = [('Summary Report', 'SYSTEM')]
    bdt_service_client.createPorfolioExposureExportSet(name, model, portfolio_list, report_list)

    date_rng = [d.strftime("%Y-%m-%d") for d in pd.date_range(start='2024-07-17', end='2024-07-17').date]
    for d in date_rng:
        print(f"Working on {d}...")
        job_id = bdt_service_client.sendExportJob(name, d, port_name + '-Exp')
        output = bdt_service_client.getPortfolioExposureReports(job_id)
    
        reports = output['Reports']
        status = output['Status']
    
        exposures = pd.DataFrame(reports[0]['Detail'])
    
        exposures.set_index(pd.Index([d for _ in range(len(exposures))]), inplace=True)
        exposures.rename(columns={"Asset ID": "AssetID", "Asset ID Type": "AssetIDType", "Factor": "Factor", "Exposure": "Exposure"}, inplace=True)
    
        print(exposures.to_sql('exposures', conn, if_exists='append',index=True, index_label="Date"))
        print("Completed.")
print("All done.")

Working on 2024-07-17...
4997
Completed.
All done.


In [76]:
cur.execute("""
SELECT max(Date) FROM exposures WHERE date < '2024-07-12' LIMIT 10
""")
print(cur.fetchall())

[('2024-03-02',)]


In [ ]:
cur.execute("""
SELECT date, count(*) FROM exposures GROUP BY Date
""")
print(cur.fetchall())

## Terminate connections

In [90]:
cur.close()
conn.close()